## Deliverable 2. Create a Customer Travel Destinations Map.

In [32]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [33]:
# jupyter nbextension enable --py gmaps
# jupyter nbextension enable --py widgetsnbextension

In [34]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,53.87,79,100,6.35,overcast clouds
1,1,Surt,LY,31.2089,16.5887,77.67,70,16,17.20,few clouds
2,2,Hermanus,ZA,-34.4187,19.2345,68.90,66,8,6.02,clear sky
3,3,Longyearbyen,SJ,78.2186,15.6401,21.04,73,20,8.05,few clouds
4,4,New Norfolk,AU,-42.7826,147.0587,47.70,65,41,1.99,scattered clouds


In [35]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [36]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,53.87,79,100,6.35,overcast clouds
1,1,Surt,LY,31.2089,16.5887,77.67,70,16,17.20,few clouds
2,2,Hermanus,ZA,-34.4187,19.2345,68.90,66,8,6.02,clear sky
6,6,Half Moon Bay,US,37.4636,-122.4286,60.10,78,40,0.00,scattered clouds
7,7,Mbaiki,CF,3.8678,17.9892,79.34,80,89,2.66,light rain
8,8,Ribeira Grande,PT,38.5167,-28.7000,73.80,73,40,18.41,scattered clouds
9,9,Hasaki,JP,35.7333,140.8333,68.40,86,36,6.93,scattered clouds
10,10,Kapaa,US,22.0752,-159.3190,75.52,75,90,5.01,light rain
11,11,Trojes,AL,41.7560,19.9620,61.29,82,75,8.23,broken clouds
13,13,Kavieng,PG,-2.5744,150.7967,80.94,80,99,8.43,light rain


In [37]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                481
City                   481
Country                477
Lat                    481
Lng                    481
Max Temp               481
Humidity               481
Cloudiness             481
Wind Speed             481
Current Description    481
dtype: int64

In [38]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Nikolskoye,RU,59.7035,30.7861,53.87,79,100,6.35,overcast clouds
1,1,Surt,LY,31.2089,16.5887,77.67,70,16,17.20,few clouds
2,2,Hermanus,ZA,-34.4187,19.2345,68.90,66,8,6.02,clear sky
6,6,Half Moon Bay,US,37.4636,-122.4286,60.10,78,40,0.00,scattered clouds
7,7,Mbaiki,CF,3.8678,17.9892,79.34,80,89,2.66,light rain


In [39]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Nikolskoye,RU,53.87,overcast clouds,59.7035,30.7861,
1,Surt,LY,77.67,few clouds,31.2089,16.5887,
2,Hermanus,ZA,68.90,clear sky,-34.4187,19.2345,
6,Half Moon Bay,US,60.10,scattered clouds,37.4636,-122.4286,
7,Mbaiki,CF,79.34,light rain,3.8678,17.9892,
8,Ribeira Grande,PT,73.80,scattered clouds,38.5167,-28.7000,
9,Hasaki,JP,68.40,scattered clouds,35.7333,140.8333,
10,Kapaa,US,75.52,light rain,22.0752,-159.3190,
11,Trojes,AL,61.29,broken clouds,41.7560,19.9620,
13,Kavieng,PG,80.94,light rain,-2.5744,150.7967,


In [40]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
# 6c. Get latitude and longitude from DataFrame
params["location"] = f"{lat},{lng}"
    
# 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

# 6e. Make request and retrieve the JSON data from the search. 
hotels = requests.get(base_url, params=params).json()
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
     

In [41]:
# 7. Drop the rows where there is no Hotel Name.
# clean_hotel_df = hotel_df.dropna()
# clean_hotel_df
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
695,Ilo,PE,61.66,overcast clouds,-17.6394,-71.3375,Torreblanca Hotel


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descriptio</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig